In [5]:
import tweepy
import re
from textblob import TextBlob
import pandas as pd
from pandas import DataFrame
import csv


import numpy as np
import joblib
import pickle
from sklearn import model_selection
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score

In [19]:
api_key = '1i8jwgi0gOPEWKmE97rZXQA75'
api_secret_key = 'GcS1hK8ptIS0zOYbtdQEIaPGPUDG8PWSLYJLdqICn6e5hxc1e5'
access_token = '1385974544972738561-akbxblNYYOyVHKkxjV3W5Ideg2jSCq'
access_token_secret = 'a5jLvkQ0CpIi0d1XChg8IZEhGWToNiKiNG5IfJDGhrS4C'

In [20]:
auth = tweepy.OAuthHandler(api_key,api_secret_key)
auth.set_access_token(access_token,access_token_secret)
api = tweepy.API(auth)

In [46]:
hasilSearch = api.search(q="vaksin",lang="en", count=8000)

In [145]:
hasil = []

In [146]:
for tweet in hasilSearch:
    tweet_properties = {}
    
    tweet_properties['tanggal_tweet'] = tweet.created_at
    tweet_properties['pengguna'] = tweet.user.screen_name
    tweet_properties['isi_tweet'] = tweet.text
    
    tweet_bersih = " ".join(re.sub("RT|:|(@[A-Za-z0-9]+)|(\w+:\/\/\S+)"," ",tweet.text).split()) #menghapus hashtag, mention dan link

    #proses case folding
    tweet_bersih = tweet_bersih.lower() #membuat text menjadi huruf kecil semua

    #proses tokenizing
    tweet_bersih = tweet_bersih.replace('\\t', " ").replace('\\n'," ").replace('\\u', " ").replace('\\', "")  #menghapus tab, new line, back slice
    tweet_bersih = tweet_bersih.encode('ascii', 'replace').decode('ascii') #menghapus non ascii
    
    analysis = TextBlob(tweet_bersih)
    
    if analysis.sentiment.polarity > 0.0:
        tweet_properties["sentimen"] = "positif"
    elif analysis.sentiment.polarity == 0.0:
        tweet_properties["sentimen"] = "netral"
    else:
        tweet_properties["sentimen"] = "negatif"
    
    if tweet.retweet_count > 0:
        if tweet_properties not in hasil:
            hasil.append(tweet_properties)
    else:
        hasil.append(tweet_properties)
    
    print(tweet_properties['isi_tweet'])
#     print(tweet_properties['isi_tweet'])

RT @melakaboy: Look. VAKSIN COVID yang paling mahal pun below RM 100 I'm sure at least 5 Million Malaysians are willing to pay for our own…
@BTSVotingIndo @BTS_twt vaksin

INDOMY HERE FOR BTS
I vote #BTSARMY  for #BestFanArmy at the 2021 #iHeartAwards @BTS_twt
Ayah orang terakhir hari ni kena vaksin sebab takut. Rupanya before kena vaksin dia punya surrounding people mostly… https://t.co/HiLiV4PDeU
Vaksin ni macam seatbelt actually .
@shimkoongnim Done my first dose last sunday. Mine is astrazeneca vaksin. Kamu?
RT @AmiKimmy: I had my first AZ jab today but i dont feel like taking a photo. Why? Earlier i stop by Petron and the workers there said “Ki…
RT @AmiKimmy: I had my first AZ jab today but i dont feel like taking a photo. Why? Earlier i stop by Petron and the workers there said “Ki…
Vaksin appointment = Done 😬
done 💉 appointment 5.30, datang 4.30, settle vaksin 5.25 😂 https://t.co/oxSRyvOMMh
Betul , vaksin tk boleh prevent 100% Covid-19..the virus become controllable if you take 

In [147]:
# hasil

In [148]:
df =  DataFrame(data = hasil)

In [149]:
df.head(5)

,tanggal_tweet,pengguna,isi_tweet,sentimen
0,2021-05-19 13:37:00,afsyad99,RT @melakaboy: Look. VAKSIN COVID yang paling ...,positif
1,2021-05-19 13:34:21,sakhayuv,@BTSVotingIndo @BTS_twt vaksin\n\nINDOMY HERE ...,netral
2,2021-05-19 13:31:35,nurthaqifah98,Ayah orang terakhir hari ni kena vaksin sebab ...,positif
3,2021-05-19 13:30:27,Mhdkmz_,Vaksin ni macam seatbelt actually .,netral
4,2021-05-19 13:29:28,jongjinsbeanie,@shimkoongnim Done my first dose last sunday. ...,positif


In [150]:
df.loc[df['sentimen'] == 'negatif'].head(10)

,tanggal_tweet,pengguna,isi_tweet,sentimen
13,2021-05-19 13:19:52,amrlhssn,RT @intnafqh_: @AdDien90 Please report this ac...,negatif
18,2021-05-19 13:08:33,acusme17,RT @intnafqh_: @AdDien90 Please report this ac...,negatif
39,2021-05-19 12:43:13,zrthaishaa,RT @intnafqh_: @AdDien90 Please report this ac...,negatif
40,2021-05-19 12:41:41,my_outdoor82,@SyafieMustafa Sadly i do have a friend like t...,negatif
47,2021-05-19 12:29:18,zulaikhajmldn,RT @intnafqh_: @AdDien90 Please report this ac...,negatif
50,2021-05-19 12:24:36,deijides,RT @intnafqh_: @AdDien90 Please report this ac...,negatif
58,2021-05-19 12:12:24,nrslnaa,RT @intnafqh_: @AdDien90 Please report this ac...,negatif
84,2021-05-19 11:41:14,mrymisml,The only sin i wish now is vaksin 🤲🏻\n- dont g...,negatif
87,2021-05-19 11:37:15,ct_shrh,RT @intnafqh_: @AdDien90 Please report this ac...,negatif
93,2021-05-19 11:33:12,murni_sha,RT @rsharminz_: @841128idayu @HausofHilton Sor...,negatif


In [151]:
df['sentimen'].value_counts(ascending=True)

negatif    12
netral     17
positif    71
Name: sentimen, dtype: int64

In [152]:
x = df['isi_tweet']
y = df['sentimen']

In [153]:
bow_transformer = CountVectorizer()
print(df['isi_tweet'].shape)
x = bow_transformer.fit_transform(df['isi_tweet'])

print(x.toarray())
print('shape of Sparse Matriks: ', x.shape)
print('Amount of Non-Zero occurrences: ', x.nnz)

filename1 = 'count_vectorized2.pkl'
pickle.dump(bow_transformer, open(filename1, 'wb'))

tf_transform = TfidfTransformer(use_idf=False).fit(x)
x = tf_transform.transform(x)
print(x.shape)

filename1 = 'tfid_transform2.pkl'
pickle.dump(tf_transform, open(filename1, 'wb'))

(100,)
[[1 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
shape of Sparse Matriks:  (100, 403)
Amount of Non-Zero occurrences:  2122
(100, 403)


In [154]:
density = (100.0 * x.nnz / (x.shape[0] * x.shape[1]))
print('Density: {}'.format((density)))

Density: 5.265508684863524


In [155]:
#splitting data
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2)


In [156]:
#Naive Bayes Method 
#classifier data
nb = MultinomialNB()
nb.fit(x_train, y_train)
preds = nb.predict(x_test)

In [157]:
print(classification_report(y_test, preds))

              precision    recall  f1-score   support

     negatif       1.00      1.00      1.00         1
      netral       0.00      0.00      0.00         3
     positif       0.84      1.00      0.91        16

    accuracy                           0.85        20
   macro avg       0.61      0.67      0.64        20
weighted avg       0.72      0.85      0.78        20



c:\users\asus\appdata\local\programs\python\python37-32\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\users\asus\appdata\local\programs\python\python37-32\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\users\asus\appdata\local\programs\python\python37-32\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(avera

In [158]:
from io import StringIO
classification = classification_report(y_test, preds)
s = StringIO(classification)
with open('classification.csv', 'w') as f:
    for line in s:
        f.write(line)

c:\users\asus\appdata\local\programs\python\python37-32\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\users\asus\appdata\local\programs\python\python37-32\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\users\asus\appdata\local\programs\python\python37-32\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(avera

In [159]:
print (accuracy_score(y_test, preds))

0.85


In [160]:
accuracy = accuracy_score(y_test,preds)
a = np.asarray([accuracy])
np.savetxt('accuracy.csv', a, delimiter=',', fmt='%s')

In [161]:
Final = df[['tanggal_tweet','pengguna','isi_tweet','sentimen']]

In [162]:
Final.to_csv('hasil_analysis.csv')

In [163]:
model = LogisticRegression()
model.fit(x_train, y_train)

LogisticRegression()

In [164]:
filename = 'model_analisis.pkl'
pickle.dump(model, open(filename, 'wb'))

In [165]:
loaded_model = pickle.load(open(filename, 'rb'))
result = loaded_model.score(x_test, y_test)
print(result)

0.85


In [166]:
loaded_model.predict(x_test)

array(['positif', 'positif', 'positif', 'positif', 'positif', 'positif',
       'negatif', 'positif', 'positif', 'positif', 'positif', 'positif',
       'positif', 'positif', 'positif', 'positif', 'positif', 'positif',
       'positif', 'positif'], dtype=object)